In [ ]:
import math
import random
import time
from jupyterthemes import jtplot
jtplot.style()

import gym
gym.logger.set_level(40)
import numpy as np

import nnabla as nn
import nnabla.logger as logger
import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solver as S
from nnabla.contrib.context import extension_context
from nnabla.monitor import Monitor, MonitorSeries, MonitorTimeElapsed

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
env = gym.make("Hopper-v2")
state_num = env.observation_space.shape[0]
action_num = env.action_space.shape[0]
hidden_size = 128
max_frames = 20000
batch_size = 32
gamma = 0.99
learning_rate = 1e-3

In [2]:
#Actor:状態に対して行動の選択確率(方策)を返す
def Actor(state):
    with nn.parameter_scope("Actor"):
        with nn.parameter_scope("Actor-affine1"):
            h = F.relu(PF.affine(state, hidden_size))
        with nn.parameter_scope("Actor-affine2"):
            A = PF.affine(h, action_num)
    return A

#Critic:状態に対して価値を返す
def Critic(state):
    with nn.parameter_scope("Critic"):
        with nn.parameter_scope("Critic-affine1"):
            h = F.relu(PF.affine(state, hidden_size))
        with nn.parameter_scope("Critic-affine2"):
            C = PF.affine(h, 1)
    return C

def get_policy(state):
    State = nn.Variable([1, state_num])
    State.d = state
    Policy = Actor(state)
    Policy.forward()
    return Policy #.d


def get_value(state):
    State = nn.Variable([1, state_num])
    State.d = state
    Value = Critic(state)
    Value.forward()
    return Value.d #.d

